# Landmark recognition with few-shot learning + ResNet50 + Decision tree
## Loading essential packages

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Loading data

In [ ]:
path = '/kaggle/input/landmark-recognition-2021/'
os.listdir(path)

In [ ]:
train_data = pd.read_csv(path+'train.csv')
train_data.head()

### Looking at the number of unique landmarks

In [ ]:
len(train_data['landmark_id'].unique())

### Props to https://www.kaggle.com/michaelscheinfeilda/landmark-recognition-2021-starter for the plot function

In [ ]:
def plot_examples(landmark_id=1):
    """ Plot 5 examples of images with the same landmark_id """
    
    fig, axs = plt.subplots(1, 5, figsize=(25, 12))
    fig.subplots_adjust(hspace = .2, wspace=.2)
    axs = axs.ravel()
    for i in range(5):
        idx = train_data[train_data['landmark_id']==landmark_id].index[i]
        image_id = train_data.loc[idx, 'id']
        file = image_id+'.jpg'
        subpath = '/'.join([char for char in image_id[0:3]])
        img = cv2.imread(path+'train/'+subpath+'/'+file)
        axs[i].imshow(img)
        axs[i].set_title('landmark_id: '+str(landmark_id))
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])
        print(path+'train/'+subpath+'/'+file)

In [ ]:
plot_examples(landmark_id=7)

## Loading necessary modules

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Flatten, Input
from tensorflow.keras.preprocessing import image

## Loading ResNet50 model

Selecting 100x100x3 as the dimension for the images to be used

In [ ]:
new_path = '../input/resnet50' 
os.listdir(new_path)

In [ ]:
base_model = ResNet50(weights='../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', pooling=max, include_top = False)
input = Input(shape=(100,100,3),name = 'image_input')
x = base_model(input)
x = Flatten()(x)
model = Model(inputs=input, outputs=x)

Example of extracting ResNet50 features from a sample image

In [ ]:
img_path = '/kaggle/input/landmark-recognition-2021/train/5/9/7/597353dfbb3df649.jpg'
img = image.load_img(img_path, target_size=(100, 100))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
features_reduce =  features.squeeze()
print(features_reduce.size)

## Function to obtain ResNet50 image features for each landmark label

Take average feature value of first 5 (if present) images

In [ ]:
def obtain_features(landmark_id=1):
    """ Obtain features of 5 examples of images with the same landmark_id """
    
    feature_vals = []
    for i in range(5):
        try:
            idx = train_data[train_data['landmark_id']==landmark_id].index[i]
            image_id = train_data.loc[idx, 'id']
            file = image_id+'.jpg'
            subpath = '/'.join([char for char in image_id[0:3]])
            img = cv2.imread(path+'train/'+subpath+'/'+file)
            #print(path+'train/'+subpath+'/'+file)
            img_path = path+'train/'+subpath+'/'+file
            img = image.load_img(img_path, target_size=(100, 100))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            features = model.predict(x)
            features_reduce =  features.squeeze()
            feature_vals.append(features_reduce)
        except:
            pass
    try:
        return sum(feature_vals)/len(feature_vals)
    except:
        return None
    

In [ ]:
feature_vals = obtain_features(landmark_id=1)
print(feature_vals)

### Seeing the labels of the first N landmarks

In [ ]:
no_of_landmarks = 100
landmark_ids = train_data['landmark_id'].unique()
print(landmark_ids[:no_of_landmarks])

### Obtaining features for the first N landmarks

In [ ]:
features_landmarks = []
for i in range(no_of_landmarks):
    if i%10 == 0:
        print(i, end=',')
    landmark = landmark_ids[i]
    features_landmarks.append(obtain_features(landmark_id=landmark))
    

In [ ]:
print(features_landmarks[0].shape)

In [ ]:
features_landmarks = np.array(features_landmarks)
print(features_landmarks.shape)

## Autoencoder to compress feature vector

In [ ]:
from tensorflow.keras import Input, Model, layers

latent_dim = 10

encoder_inputs = Input(shape=(32768,))
x = layers.Dense(4000, activation="relu")(encoder_inputs)
x = layers.Dense(100, activation="relu")(x)
z = layers.Dense(latent_dim, name="z_mean")(x)
encoder = Model(encoder_inputs, z, name="encoder")
encoder.summary()

In [ ]:
latent_inputs = Input(shape=(latent_dim,))
x = layers.Dense(100, activation="relu")(latent_inputs)
x = layers.Dense(400, activation="relu")(x)
decoder_outputs = layers.Dense(32768, activation="relu")(x)
decoder = Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

In [ ]:
latent_dim = 10

class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = encoder
    self.decoder = decoder

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder(latent_dim)

In [ ]:
from tensorflow.keras import metrics, losses

autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [ ]:
autoencoder.fit(features_landmarks, features_landmarks,
                epochs=30,
                shuffle=True)

In [ ]:
'''test_landmark = 36
test_index = 2
idx = train_data[train_data['landmark_id']==test_landmark].index[test_index]
image_id = train_data.loc[idx, 'id']
file = image_id+'.jpg'
subpath = '/'.join([char for char in image_id[0:3]])
img = cv2.imread(path+'train/'+subpath+'/'+file)
#print(path+'train/'+subpath+'/'+file)
img_path = path+'train/'+subpath+'/'+file
img = image.load_img(img_path, target_size=(100, 100))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
features = model.predict(x)
features_reduce =  np.array(features.squeeze())
features_reduce =  np.reshape(features_reduce, (1,32768))
encoded_img = autoencoder.encoder(features_reduce).numpy()
print(encoded_img)'''

In [ ]:
encoded_features = []
i = 0
for elem in features_landmarks:
    if i % 100 == 0:
        print(i, end=',')
    elem =  np.reshape(elem, (1,32768))
    encoded_val = autoencoder.encoder(elem).numpy()
    encoded_features.append(encoded_val[0])
    i += 1

## Fitting a decision tree on the landmark labels and corresponding average features

In [ ]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(random_state=1)

# Fit the model
decision_tree.fit(encoded_features, landmark_ids[:no_of_landmarks])

## Code to check prediction for individual predicted labels

### Check within training data (sanity check)

In [ ]:
'''test_landmark = 36
test_index = 2
idx = train_data[train_data['landmark_id']==test_landmark].index[test_index]
image_id = train_data.loc[idx, 'id']
file = image_id+'.jpg'
subpath = '/'.join([char for char in image_id[0:3]])
img = cv2.imread(path+'train/'+subpath+'/'+file)
#print(path+'train/'+subpath+'/'+file)
img_path = path+'train/'+subpath+'/'+file
img = image.load_img(img_path, target_size=(100, 100))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
features = model.predict(x)
features_reduce =  features.squeeze()
features_reduce =  np.reshape(features_reduce, (1,32768))
encoded_img = autoencoder.encoder(features_reduce).numpy()
predictions = decision_tree.predict([encoded_img[0]])
print(predictions)'''

### Check within test data

In [ ]:
'''test_img = '../input/landmark-recognition-2021/test/9/0/0/900bb54db718a992.jpg'
img = image.load_img(img_path, target_size=(100, 100))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
features_reduce =  features.squeeze()

features_reduce =  np.reshape(features_reduce, (1,32768))
encoded_img = autoencoder.encoder(features_reduce).numpy()
predictions = decision_tree.predict([encoded_img[0]])
print(predictions)'''

## Load test data in submission format

In [ ]:
test_data = pd.read_csv(path+'sample_submission.csv')
test_data.head()

## Obtain predictions of test data

In [ ]:
    predictions = []
    for i in range(len(test_data)):
        try:
            if i%100==0:
                print(i, end=',')
            image_id = test_data.loc[i, 'id']
            file = image_id+'.jpg'
            subpath = '/'.join([char for char in image_id[0:3]])
            img = cv2.imread(path+'test/'+subpath+'/'+file)
            #print(path+'test/'+subpath+'/'+file)
            img_path = path+'test/'+subpath+'/'+file
            img = image.load_img(img_path, target_size=(100, 100))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            features = model.predict(x)
            features_reduce =  features.squeeze()
            features_reduce =  np.reshape(features_reduce, (1,32768))
            encoded_img = autoencoder.encoder(features_reduce).numpy()
        except:
            pass
        pred = decision_tree.predict([encoded_img[0]])
        #print(prediction)
        predictions.append(str(pred[0]) + ' 1.00000000')

## Convert to submission format

In [ ]:
output = pd.DataFrame({'id': test_data.id,
                       'landmarks': predictions})

output.head()
output.to_csv('submission.csv', index=False)